In [2]:
import polars as pl
from datetime import timedelta, datetime, date

In [19]:
test_df = pl.DataFrame(
    {
        "date": [
            "2020-08-01",
            "2020-08-02",
            "2020-08-03",
            "2020-08-04",
            "2020-08-05",
        ],
        "value": [1, 2, 3, 4, 5],
    }
)


In [22]:
test_df = test_df.with_columns(pl.col("date").cast(pl.Date).cast(pl.Datetime("ms")).alias("date"), pl.int_range(1, pl.len() + 1).alias("count"))


In [23]:
test_df.select(
    (pl.col("date") + timedelta(days=1, minutes=1)).alias("date"), pl.int_range(1, pl.len() + 1).alias("c")
    )

date,c
datetime[ms],i64
2020-08-02 00:01:00,1
2020-08-03 00:01:00,2
2020-08-04 00:01:00,3
2020-08-05 00:01:00,4
2020-08-06 00:01:00,5


In [25]:
test_df.select(
    (pl.col("date") + timedelta(days=1, minutes=1)).alias("date"), pl.int_range(1, pl.len() + 1).alias("c")
    ).set_sorted("date").join_asof(
        test_df.set_sorted("date"),
        on="date",
        strategy="backward"
        )

date,c,value,count
datetime[ms],i64,i64,i64
2020-08-02 00:01:00,1,2,2
2020-08-03 00:01:00,2,3,3
2020-08-04 00:01:00,3,4,4
2020-08-05 00:01:00,4,5,5
2020-08-06 00:01:00,5,5,5


In [30]:
test_df.with_columns(shift=pl.Series("t", [1, 2, 0, 1, 2])).with_columns(pl.col("date").shift(pl.col("shift")).alias("date_2"))

ComputeError: n must be a single value.